In [2]:
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
sc=SparkContext()
spark=SparkSession(sc)

## Load the Data

In [3]:
raw_train=spark.read.csv('titanic_train.csv',inferSchema=True,header=True)
raw_train.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [4]:
raw_test=spark.read.csv('titanic_test.csv',inferSchema=True,header=True)
raw_test.show(5)

+-----------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------+
|PassengerId|Pclass|                Name|   Sex| Age|SibSp|Parch| Ticket|   Fare|Cabin|Embarked|
+-----------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------+
|        892|     3|    Kelly, Mr. James|  male|34.5|    0|    0| 330911| 7.8292| null|       Q|
|        893|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0| 363272|    7.0| null|       S|
|        894|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0| 240276| 9.6875| null|       Q|
|        895|     3|    Wirz, Mr. Albert|  male|27.0|    0|    0| 315154| 8.6625| null|       S|
|        896|     3|Hirvonen, Mrs. Al...|female|22.0|    1|    1|3101298|12.2875| null|       S|
+-----------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------+
only showing top 5 rows



## Analyse the data

In [6]:
raw_train.describe().toPandas()

/Users/shivaganesh/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/shivaganesh/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


,summary,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,count,891,891,891,891,891,714,891,891,891,891,204,889
1,mean,446.0,0.3838383838383838,2.308641975308642,None,None,29.69911764705882,0.5230078563411896,0.38159371492704824,260318.54916792738,32.2042079685746,None,None
2,stddev,257.3538420152301,0.48659245426485753,0.8360712409770491,None,None,14.526497332334035,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,None,None
3,min,1,0,1,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""",female,0.42,0,0,110152,0.0,A10,C
4,max,891,1,3,"van Melkebeke, Mr. Philemon",male,80.0,8,6,WE/P 5735,512.3292,T,S


In [7]:
#check for null values
raw_train.select([eval('raw_train.'+x+'.isNull().cast("int").alias("' +x+ '")') for x in raw_train.columns]).show(10)

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    1|       0|
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    0|       0|
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    1|       0|
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    0|       0|
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    1|       0|
|          0|       0|     0|   0|  0|  1|    0|    0|     0|   0|    1|       0|
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    0|       0|
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    1|       0|
|          0|       0|     0|   0|  0|  0|    0|    0|     0|   0|    1|       0|
|          0|   

In [8]:
raw_train.select(raw_train.Cabin.isNull().cast('int').alias('Cabin Null')).groupBy().sum().toPandas()

,sum(Cabin Null)
0,687


In [9]:
#null values in train data
raw_train.select([eval('raw_train.'+x+'.isNull().cast("int").alias("' +x+ '")') for x in raw_train.columns]) \
.groupBy().sum().toPandas()

,sum(PassengerId),sum(Survived),sum(Pclass),sum(Name),sum(Sex),sum(Age),sum(SibSp),sum(Parch),sum(Ticket),sum(Fare),sum(Cabin),sum(Embarked)
0,0,0,0,0,0,177,0,0,0,0,687,2


In [10]:
#null values in test data
raw_test.select([eval('raw_test.' +x+'.isNull().cast("int").alias("' +x+ '")') for x in raw_test.columns]) \
.groupBy().sum().toPandas()

,sum(PassengerId),sum(Pclass),sum(Name),sum(Sex),sum(Age),sum(SibSp),sum(Parch),sum(Ticket),sum(Fare),sum(Cabin),sum(Embarked)
0,0,0,0,0,86,0,0,0,1,327,0


In [11]:
raw_test.count()

418

In [12]:
raw_train.count()

891

In [13]:
#no of Passengers survived
raw_train.groupBy('Survived').count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [14]:
raw_train.groupBy('Sex','Survived').count().show()

+------+--------+-----+
|   Sex|Survived|count|
+------+--------+-----+
|  male|       0|  468|
|female|       1|  233|
|female|       0|   81|
|  male|       1|  109|
+------+--------+-----+



In [15]:
raw_train.crosstab('Embarked','Survived').show()

+-----------------+---+---+
|Embarked_Survived|  0|  1|
+-----------------+---+---+
|                S|427|217|
|                C| 75| 93|
|                Q| 47| 30|
|             null|  0|  2|
+-----------------+---+---+



In [16]:
raw_train.crosstab('Sex','Survived').show()

+------------+---+---+
|Sex_Survived|  0|  1|
+------------+---+---+
|        male|468|109|
|      female| 81|233|
+------------+---+---+



In [17]:
raw_train=raw_train.fillna({'Embarked':'S'})

In [18]:
raw_train.groupBy('Embarked').count().show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   77|
|       C|  168|
|       S|  646|
+--------+-----+



In [19]:
#drop Cabin feature as it has many null values
train=raw_train.drop('Cabin')

In [20]:
train.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = false)



In [21]:
from pyspark.sql import functions as f

In [25]:
mean_age=raw_train.select(f.mean(f.col('Age'))).collect()[0][0]

In [28]:
train=train.withColumn('Age',f.when(f.col('Age').isNull(),mean_age).otherwise(train['Age']))

In [29]:
train.show()

+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|              Age|SibSp|Parch|          Ticket|   Fare|Embarked|
+-----------+--------+------+--------------------+------+-----------------+-----+-----+----------------+-------+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|             22.0|    1|    0|       A/5 21171|   7.25|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|             38.0|    1|    0|        PC 17599|71.2833|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|             26.0|    0|    0|STON/O2. 3101282|  7.925|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|             35.0|    1|    0|          113803|   53.1|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|             35.0|    0|    0|          373450|   8.05|       S|
|          6|       0|  

In [30]:
train.groupBy('Embarked').count().sort(f.col('count').desc()).show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  646|
|       C|  168|
|       Q|   77|
+--------+-----+



## String Indexer

In [36]:
from pyspark.ml.feature import StringIndexer
string_indexer=StringIndexer(inputCol='Embarked',outputCol='index_embarked')
train=string_indexer.fit(train).transform(train)

In [41]:
train.toPandas()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,index_embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,S,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C,1.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,S,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,S,0.0
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,S,0.0
5,6,0,3,"Moran, Mr. James",male,29.699118,0,0,330877,8.4583,Q,2.0
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.000000,0,0,17463,51.8625,S,0.0
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.000000,3,1,349909,21.0750,S,0.0
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.000000,0,2,347742,11.1333,S,0.0
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.000000,1,0,237736,30.0708,C,1.0
